## 1. Построение дерева решений

In [5]:
import numpy as np
import pandas as pd

Возьмем задачу предсказания наличия диабета по результатам медицинских анализов.

In [6]:
data = pd.read_csv(r'C:\Users\Lenovo\Documents\diabetes.csv')

In [7]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Поделим выборку на обучение и контроль с помощью функции `train_test_split`.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

Реализация решающего дерева представлена в модуле `sklearn.tree`. Для решения задачи классификации воспользуемся функцией `DecisionTreeClassifier` из данного модуля.

In [11]:
from sklearn.tree import DecisionTreeClassifier

Для обучения дерева решений необходимо поместить его в отдельную переменную, после чего применить метод `fit` к обучающей выборке.

In [12]:
clf = DecisionTreeClassifier(random_state=42)

In [13]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Оценим качество модели с помощью метрики `accuracy`.

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
y_pred = clf.predict(X_test)

In [16]:
accuracy_score(y_test, y_pred)

0.70995670995671

## 2. Визуализация

Для визуализации деревьев решений можно воспользоваться функцией `export_graphviz`, также находящейся в модуле `sklearn.tree`.

In [17]:
from sklearn.tree import export_graphviz

In [18]:
export_graphviz(clf, out_file='decision_tree.dot', 
                filled=True,
                feature_names=X.columns,
                class_names=['0','1'])

Для визуализации полученного дерева решений воспользуемся сервисом http://www.webgraphviz.com/.

В папке, в которой находится данный ноутбук, появится файл decision_tree.dot. Его можно открыть с помощью текстового редактора, скопировать содержимое, поместить содержимое в текстовое поле на webgraphviz и нажать на "Generate Graph!".

In [19]:
# from sklearn import tree

# import matplotlib.pyplot as plt
# %matplotlib inline

In [20]:
# plt.figure(figsize=(12,12))

# tree.plot_tree(clf,
#                feature_names=X.columns, 
#                class_names=['0', '1'],
#                filled=True)
# plt.show()

## 3. Выбор гиперпараметров

Среди гиперпараметров дерева решений полезными являются:
- `class_weight` $-$ отвечает за сбалансированность классов (дает больший штраф редким классам);
- `max_depth` $-$ отвечает за глубину дерева, что позволяет избежать переобучения;
- `max_features` $-$ позволяет задавать количество признаков, рассматриваемых при выборе предиката;
- `min_samples_leaf` $-$ минимальное количество объектов в терминальной вершине.

Для выбора оптимального набора гиперпараметров часто используется полный перебор, реализованный в функции `GridSearchCV`. `GridSearchCV` принимает на вход модель (`estimator`), сетку параметров (`param_grid`), генератор разбиений (`cv`), а также метрику качества, которую необходимо оптимизировать (`scoring`). Данная функция методом кросс-валидации рассматривает каждый возможный набор гиперпараметров и возвращает оптимальный с т.з. выбранной метрики.

In [21]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

В качестве генератора разбиений возьмем `StratifiedKFold`, чтобы гарантировать сбалансированность классов в каждом из блоков обучения и контроля.

In [22]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [23]:
tree_params = {'class_weight': ['balanced', None],
               'max_depth': [3,5,7,9],
               'max_features': [3,5,7]}

In [24]:
grid_search = GridSearchCV(clf, tree_params, scoring='accuracy', cv=skf)

In [25]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [3, 5, 7, 9], 'max_features': [3, 5, 7]},
             scoring='accuracy')

In [26]:
grid_search.best_params_

{'class_weight': None, 'max_depth': 3, 'max_features': 7}

In [27]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=3, max_features=7, random_state=42)

In [28]:
grid_search.best_estimator_.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, max_features=7, random_state=42)

In [29]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [30]:
accuracy_score(y_test, y_pred)

0.7662337662337663